In [1]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_datasets
from transformers import *
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow import keras

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tqdm.notebook import tqdm
# from wandb.keras import WandbCallback
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
# !wget https://raw.githubusercontent.com/wangz10/contrastive_loss/master/losses.py
import losses
from losses import max_margin_contrastive_loss
import feed
from feed import *
import time
import model
from model import *

2.2.0


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


2.2.0


In [2]:
train_df = pd.read_csv('files/train.csv')
val_df = pd.read_csv('files/val.csv')
test_df = pd.read_csv('files/test.csv')
pred_df = pd.read_csv('files/pred.csv')
Num_class = len(set(train_df.label.value_counts()))
print(Num_class)
print(train_df.label_id.value_counts())

6
3    5354
2    3601
4     686
0     193
1      93
5      73
Name: label_id, dtype: int64


In [3]:
# Load dataset, tokenizer, model from pretrained model/vocabulary
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
# model = TFAlbertForTextClassification.from_pretrained('albert-base-v2', num_labels=Num_class, isSCR=True)
model = TFAlbertTextClassification.from_pretrained('albert-base-v2',num_labels=Num_class, isSCR=True)
# print(model)
# print(type(model))

inputs : ()
kwargs : {'isSCR': True}
{}


Some weights of the model checkpoint at albert-base-v2 were not used when initializing TFAlbertTextClassification: ['predictions']
- This IS expected if you are initializing TFAlbertTextClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFAlbertTextClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFAlbertTextClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['dropout_3', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
print(model)

In [5]:
# Convert data to InputExample format

train_examples = convert_text_to_examples(train_df['text'], train_df['label_id'])
val_examples = convert_text_to_examples(val_df['text'], val_df['label_id'])
test_examples = convert_text_to_examples(test_df['text'], test_df['label_id'])
pred_examples = convert_text_to_examples(pred_df['text'], np.zeros(len(pred_df) ))


In [6]:
# Convert to features
max_seq_length = 128
(train_input_ids, train_input_masks, train_segment_ids, train_labels ) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(val_input_ids, val_input_masks, val_segment_ids, val_labels ) = convert_examples_to_features(tokenizer, val_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels ) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)
(pred_input_ids, pred_input_masks, pred_segment_ids, pred_labels ) = convert_examples_to_features(tokenizer, pred_examples, max_seq_length=max_seq_length)

In [7]:
BATCH_SIZE=32
train_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": tf.constant( train_input_ids, dtype=tf.int32),
                                                  "attention_mask": tf.constant( train_input_masks, dtype=tf.int32),
                                                  "token_type_ids": tf.constant( train_segment_ids, dtype=tf.int32),}
                                                    ,tf.constant(train_labels)))
train_dataset = train_dataset.shuffle(100, reshuffle_each_iteration=True).batch(BATCH_SIZE,drop_remainder=True).repeat(2)

val_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": tf.constant( val_input_ids, dtype=tf.int32),
                                                  "attention_mask": tf.constant( val_input_masks, dtype=tf.int32),
                                                  "token_type_ids": tf.constant( val_segment_ids, dtype=tf.int32),}
                                                    ,tf.constant(val_labels)))
val_dataset = val_dataset.batch(BATCH_SIZE,drop_remainder=False)

test_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": tf.constant( test_input_ids, dtype=tf.int32),
                                                  "attention_mask": tf.constant( test_input_masks, dtype=tf.int32),
                                                  "token_type_ids": tf.constant( test_segment_ids, dtype=tf.int32),}
                                                    ,tf.constant(test_labels)))
test_dataset = test_dataset.batch(BATCH_SIZE,drop_remainder=False)

In [15]:
# def max_margin_contrastive_loss(z, y, margin=1.0, metric='euclidean'):
#     '''
#     Wrapper for the maximum margin contrastive loss (Hadsell et al. 2006)
#     `tfa.losses.contrastive_loss`
#     Args:
#         z: hidden vector of shape [bsz, n_features].
#         y: ground truth of shape [bsz].
#         metric: one of ('euclidean', 'cosine')
#     '''
#     # compute pair-wise distance matrix
#     if metric == 'euclidean':
#         D = pdist_euclidean(z)
#     elif metric == 'cosine':
#         D = 1 - tf.matmul(z, z, transpose_a=False, transpose_b=True)
#     # convert squareform matrix to vector form
#     d_vec = square_to_vec(D)
#     # make contrastive labels
#     y_contrasts = get_contrast_batch_labels(y)
#     loss = tfa.losses.contrastive_loss(y_contrasts, d_vec, margin=margin)
#     # exploding/varnishing gradients on large batch?
#     return tf.reduce_mean(loss)

In [16]:
def my_loss_fn(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)  # Note the `axis=-1`


# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# scr_loss = max_margin_contrastive_loss(z, y, margin=1.0, metric='euclidean')
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=max_margin_contrastive_loss, metrics=[metric])
print(model.summary())


Model: "tf_albert_text_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
albert (TFAlbertMainLayer)   multiple                  11683584  
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  4614      
Total params: 11,688,198
Trainable params: 11,688,198
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
# Train and evaluate using tf.keras.Model.fit()
# history = model.fit(train_dataset, epochs=2, steps_per_epoch=115,
history = model.fit(train_dataset, epochs=2,
                    validation_data=val_dataset, validation_steps=7)


Epoch 1/2


NameError: in user code:

    /home/beomgon2/huggingface/hugging-venv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/beomgon2/huggingface/hugging-venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/beomgon2/huggingface/hugging-venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/beomgon2/huggingface/hugging-venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/beomgon2/huggingface/hugging-venv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_loss